## Implementation of RBF networks and test on adversarial examples

In [36]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from keras.layers import Layer
from keras import backend as K


In [37]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [38]:
def get_adversial_exemple(model, image, label, eps=0.25):
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, 0)
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.binary_crossentropy(label, prediction)
    gradient = tape.gradient(loss, image)
    signed_grad = tf.sign(gradient)
    adv_ex = image + eps*signed_grad    
    adv_ex = tf.clip_by_value(adv_ex, 0, 1)
    return adv_ex

In [29]:



class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
#         print(input_shape)
#         print(self.units)
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff, 2), axis=1)
        return K.exp(-1 * self.gamma * l2)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [30]:
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.losses import binary_crossentropy
RBF_Shallow = Sequential()
RBF_Shallow.add(Flatten(input_shape=(28, 28)))
RBF_Shallow.add(RBFLayer(10, 0.5))
RBF_Shallow.add(Dense(1, activation='sigmoid', name='foo'))

RBF_Shallow.compile(optimizer='adam', loss="binary_crossentropy",metrics=['accuracy'])


In [31]:
RBF_Shallow.fit(ds_train, batch_size=256, epochs=3)
RBF_Shallow.evaluate(ds_test)

Epoch 1/3
469/469 [==============================] - 3s 5ms/step - loss: -0.2170 - accuracy: 0.1124
Epoch 2/3
469/469 [==============================] - 3s 5ms/step - loss: -1.9915 - accuracy: 0.1124
Epoch 3/3
79/79 [==============================] - 0s 3ms/step - loss: -4.5293 - accuracy: 0.1135


[-4.529333591461182, 0.11349999904632568]

In [32]:
# convert tf.data.Dataset to numpy array
x_test = np.concatenate([x for x, y in ds_test], axis=0)
y_test = np.concatenate([y for x, y in ds_test], axis=0)


In [35]:
x_test_adv = [get_adversial_exemple(RBF_Shallow, x, y,eps=0.25).numpy()[0] for x,y in zip(x_test, y_test)]
x_test_adv = np.array(x_test_adv)

l,a =  RBF_Shallow.evaluate(x_test_adv, y_test)
c = np.mean(np.max(RBF_Shallow.predict(x_test_adv), axis=1))
print(f"Accuracy on adversarial examples: {a} with a mean confidence of {c}", )

ValueError: `logits` and `labels` must have the same shape, received ((1, 1) vs ()).